In [1]:
from __future__ import print_function
import pickle
import os.path
from googleapiclient.discovery import build
from google_auth_oauthlib.flow import InstalledAppFlow
from google.auth.transport.requests import Request
import pandas as pd
import numpy as np
import re

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

## Obtain data from Google spreadsheet

In [2]:
SCOPES = ['https://www.googleapis.com/auth/spreadsheets.readonly']

if os.path.exists('token.pickle'):
    with open('token.pickle', 'rb') as token:
        creds = pickle.load(token)
        
# If there are no (valid) credentials available, let the user log in.
if not creds or not creds.valid:
    if creds and creds.expired and creds.refresh_token:
        creds.refresh(Request())
    else:
        flow = InstalledAppFlow.from_client_secrets_file(
            'credentials.json', SCOPES)
        creds = flow.run_local_server(port=0)
    # Save the credentials for the next run
    with open('token.pickle', 'wb') as token:
        pickle.dump(creds, token)

service = build('sheets', 'v4', credentials=creds)

sheet = service.spreadsheets()

In [3]:
def gsheet2df(gsheet):
    """ Converts Google sheet data to a Pandas DataFrame.
    Note: This script assumes that your data contains a header file on the first row!
    Also note that the Google API returns 'none' from empty cells - in order for the code
    below to work, you'll need to make sure your sheet doesn't contain empty cells,
    or update the code to account for such instances.
    """
    header = gsheet.get('values', [])[0]   # Assumes first line is header!
    values = gsheet.get('values', [])[1:]  # Everything else is data.
    if not values:
        print('No data found.')
    else:
        all_data = []
        for col_id, col_name in enumerate(header):
            column_data = []
            for row in values:
                column_data.append(row[col_id])
            ds = pd.Series(data=column_data, name=col_name)
            all_data.append(ds)
        df = pd.concat(all_data, axis=1)
        return df

In [4]:
spreadsheet = '1EHTQy7nnT0JWgiWenlgxbGOpTN0jvc3ZWMSYVcb3reQ'
sheet_range = 'Sheet1!A:XX'

survey_responses = sheet.values().get(spreadsheetId=spreadsheet, range=sheet_range).execute()

In [5]:
survey = gsheet2df(survey_responses)

## Combine columns 

In [6]:
questions = ['NameoftheDistrict', '1_1', '1_2', '2_1', '2_2', '2_3', 
             '3_1.mgnrega', '3_1.pmKisan', '3_1.ujjwala', '3_1.janDhan', '3_1.oldAgeWidow', 
             '3_1.registeredLabour', '3_1.freeRation', '3_1.notEligible', '3_1.none',
             '3_2.mgnrega', '3_2.pmKisan', '3_2.ujjwala', '3_2.janDhan', '3_2.oldAgeWidow', 
             '3_2.registeredLabour', '3_2.bplRation', '3_2.generalRation', '3_2.none',
             '3_3', '3_4', '4_1', '4_2']

for question in questions:
    cols = [col for col in survey.columns if question in col] 
    if question == 'NameoftheDistrict':
        cols = cols + ['locationBihar']
    survey[question] = survey[cols].sum(1)
    survey = survey.drop(cols, 1)

cols = ['_2' , '4_2']
survey['4_2'] = survey[cols].sum(1)
survey = survey.drop(['_2'], 1)

## Clean strings

In [7]:
# Column names
survey = survey.rename(columns={"_id": "ID", "created": "time_created", "modified": "time_modified",
                                "MOB_ID": "mob_id", "introductionCallConnected": "call_connected",
                                "introductionAvailability": "availability", "locationSelectState": "state",
                                "locationNameofArea": "sub_district", "endCallStatusEndCallStatus": "end_call_status", 
                                "NameoftheDistrict": "district", "1_1": "need_meds", "1_2": "comments_health",
                                "2_1": "have_food", "2_3": "comments_food", '3_1.mgnrega': "received_mgnrega", 
                                '3_1.pmKisan': "received_pm_kisan", '3_1.ujjwala': "received_ujjwala", 
                                '3_1.janDhan': "received_jan_dhan", '3_1.oldAgeWidow': "received_old_age_widow", 
                                '3_1.registeredLabour': "received_registered_labor", 
                                '3_1.freeRation': "received_free_ration", '3_1.notEligible': "received_not_eligible", 
                                '3_1.none': "received_none", '3_2.mgnrega': "eligible_mgnrega",
                                '3_2.pmKisan': "eligible_pm_kisan", '3_2.ujjwala': "eligible_ujjwala", 
                                '3_2.janDhan': "eligible_jan_dhan", '3_2.oldAgeWidow': "eligible_old_age_widow",
                                '3_2.registeredLabour': "eligible_registered_labor", 
                                '3_2.bplRation': "eligible_bpl_ration", 
                                '3_2.generalRation': "eligible_general_ration",
                                '3_2.none': "eligible_none", "3_3": "employment_status", "3_4": "comments_benefits",
                                "4_1": "need_to_revert",
                                "4_2": "comments_additional"})

In [8]:
# yes / no / true / false etc

binary_cols = ['call_connected', 'availability', 'need_meds', 'have_food', 'received_mgnrega', 'received_pm_kisan', 
                'received_ujjwala', 'received_jan_dhan', 'received_old_age_widow', 'received_registered_labor', 
                'received_free_ration', 'received_not_eligible', 'received_none', 'eligible_mgnrega',
                'eligible_pm_kisan', 'eligible_ujjwala', 'eligible_jan_dhan', 'eligible_old_age_widow', 
                'eligible_registered_labor', 'eligible_bpl_ration', 'eligible_general_ration', 'eligible_none',
                'need_to_revert']

survey[binary_cols] = survey[binary_cols].replace(to_replace=r'(?i)^(yes|true)', value=True, regex=True)
survey[binary_cols] = survey[binary_cols].replace(to_replace=r'(?i)^(no|false)', value=False, regex=True)

In [9]:
# date columns
survey['time_created'] = pd.to_datetime(survey['time_created'], format='%Y-%m-%d %I:%M %p UTC')
survey['time_created'] = pd.to_datetime(survey['time_created'].dt.strftime('%Y-%m-%d %H:%M'))
survey['time_modified'] = pd.to_datetime(survey['time_modified'], format='%Y-%m-%d %I:%M %p UTC')
survey['time_modified'] = pd.to_datetime(survey['time_modified'].dt.strftime('%Y-%m-%d %H:%M'))

In [10]:
# state, district, sub district
def capitalizeName(name):
    name = re.sub(r"(?i)village","", name)
    name = re.sub(r"(?<=\w)([A-Z])", r" \1", name).capitalize()
    name = name.replace("-", " ")
    name = name.strip()
    name = name.title()
    return name

survey[['state', 'district', 'sub_district']] = survey[['state', 'district', 'sub_district']].applymap(capitalizeName)

In [11]:
# employment status
survey['employment_status'] = survey['employment_status'].apply(lambda x: re.sub("([^\x00-\x7F])+","",x))

In [12]:
# NaNs
survey.replace(r'^\s*$', np.NaN, regex=True, inplace=True)
survey.dropna(axis=1, how="all", inplace=True)

In [13]:
# duplicates
survey.drop_duplicates(inplace=True)

## District stats

In [15]:
survey = survey[survey['availability'] == True]

In [14]:
column_names = ['state', 'district', 'date', 'num_surveyed', 'perc_unemplyed', 'perc_unemployed_post_lockdown',
                'perc_have_jan_dhan', 'perc_need_med', 'perc_need_food', 'perc_no_benefit', 'perc_mgnrega', 
                'perc_pm_kisan', 'perc_ujjwala', 'perc_jan_dhan', 'perc_old_age_widow_disability',
                'perc_registered_labor', 'perc_free_ration']
district_dashboard = pd.DataFrame(columns = column_names)

In [16]:
states = survey['state'].unique().tolist()
states = [state for state in states if str(state) != 'nan']

In [17]:
for state in states:
    survey_state = survey[survey['state'] == state]
    districts = survey_state['district'].unique().tolist()
    districts = [district for district in districts if str(district) != 'nan']
    for district in districts:
        dist_df = survey_state[survey_state['district'] == district]
        date = dist_df['time_created'].max().strftime("%d/%m/%Y")
        num_surveyed = len(dist_df.shape)
        perc_unemployed = len(dist_df[(dist_df['employment_status'] == 'unemployed') | (dist_df['employment_status'] == 'unemployedPostLockdown')]) / num_surveyed
        perc_unemployed_post_lockdown = len(dist_df[dist_df['employment_status'] =='unemployedPostLockdown']) / num_surveyed
        perc_have_jan_dhan = len(dist_df[dist_df['eligible_jan_dhan'] == True]) / num_surveyed
        perc_need_med = len(dist_df[dist_df['need_meds'] == True]) / num_surveyed
        perc_need_food = len(dist_df[dist_df['have_food'] == False]) / num_surveyed
        perc_no_benefit = len(dist_df[dist_df['received_none'] == True]) / num_surveyed
        
        perc_mgnrega = np.NaN
        perc_pm_kisan = np.NaN
        perc_ujjwala = np.NaN
        perc_jan_dhan = np.NaN
        perc_old_age_widow_disability = np.NaN
        perc_registered_labor = np.NaN
        perc_free_ration = np.NaN
        
        if (len(dist_df[dist_df['eligible_mgnrega'] == True]) > 0):
            perc_mgnrega = len(dist_df[dist_df['received_mgnrega'] == True]) / len(dist_df[dist_df['eligible_mgnrega'] == True])
            
        if (len(dist_df[dist_df['eligible_pm_kisan'] == True]) > 0):
            perc_pm_kisan = len(dist_df[dist_df['received_pm_kisan'] == True]) / len(dist_df[dist_df['eligible_pm_kisan'] == True])
        
        if (len(dist_df[dist_df['eligible_ujjwala'] == True]) > 0):
            perc_ujjwala = len(dist_df[dist_df['received_ujjwala'] == True]) / len(dist_df[dist_df['eligible_ujjwala'] == True])
        
        if (len(dist_df[dist_df['eligible_jan_dhan'] == True]) > 0):
            perc_jan_dhan = len(dist_df[dist_df['received_jan_dhan'] == True]) / len(dist_df[dist_df['eligible_jan_dhan'] == True])
        
        if (len(dist_df[dist_df['eligible_old_age_widow'] == True]) > 0):
            perc_old_age_widow_disability = len(dist_df[dist_df['received_old_age_widow'] == True]) / len(dist_df[dist_df['eligible_old_age_widow'] == True])
        
        if (len(dist_df[dist_df['eligible_registered_labor'] == True]) > 0):
            perc_registered_labor = len(dist_df[dist_df['received_registered_labor'] == True]) / len(dist_df[dist_df['eligible_registered_labor'] == True])
        
        if (len(dist_df[(dist_df['eligible_bpl_ration'] == True) | (dist_df['eligible_general_ration'] == True)]) > 0):
            perc_free_ration = len(dist_df[dist_df['received_free_ration'] == True]) / len(dist_df[(dist_df['eligible_bpl_ration'] == True) | (dist_df['eligible_general_ration'] == True)])
        
        district_dashboard = district_dashboard.append(pd.Series([state, district, date, num_surveyed, perc_unemployed, 
                 perc_unemployed_post_lockdown, perc_have_jan_dhan, perc_need_med, perc_need_food, 
                 perc_no_benefit, perc_mgnrega, perc_pm_kisan, perc_ujjwala, perc_jan_dhan, 
                 perc_old_age_widow_disability, perc_registered_labor, perc_free_ration], 
                 index=district_dashboard.columns ), ignore_index=True)

In [18]:
district_dashboard

,state,district,date,num_surveyed,perc_unemplyed,perc_unemployed_post_lockdown,perc_have_jan_dhan,perc_need_med,perc_need_food,perc_no_benefit,perc_mgnrega,perc_pm_kisan,perc_ujjwala,perc_jan_dhan,perc_old_age_widow_disability,perc_registered_labor,perc_free_ration
0,Rajasthan,Tonk,06/04/2020,2,0.0,0.0,0.0,0.0,0.0,0.5,NaN,NaN,NaN,NaN,NaN,NaN,0.000000
1,Rajasthan,Ajmer,06/04/2020,2,0.5,0.0,0.0,0.0,0.0,0.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Rajasthan,Alwar,06/04/2020,2,0.5,0.0,0.5,0.0,0.0,0.5,0.0,NaN,NaN,0.0,NaN,NaN,0.000000
3,Rajasthan,Dausa,06/04/2020,2,0.0,0.0,0.5,0.0,0.0,0.5,NaN,0.0,NaN,0.0,NaN,NaN,0.000000
4,Karnataka,Uttara Kannada,06/04/2020,2,0.0,0.0,0.5,0.0,0.0,0.5,NaN,NaN,NaN,0.0,NaN,NaN,0.000000
5,Karnataka,Bangalore Rural,06/04/2020,2,0.0,0.0,0.0,0.0,0.0,0.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,Karnataka,Dakshina,06/04/2020,2,0.0,0.0,0.5,0.0,0.0,0.0,NaN,NaN,NaN,0.0,NaN,NaN,1.000000
7,Maharashtra,Amravati,06/04/2020,2,0.5,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,0.000000
8,Chhattisgarh,Balod,06/04/2020,2,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.5,NaN,NaN,1.333333
9,Chhattisgarh,Raipur,06/04/2020,2,0.5,0.5,0.5,0.5,0.5,1.0,NaN,NaN,NaN,0.0,NaN,NaN,NaN


## State stats

In [20]:
column_names = ['state', 'date', 'num_surveyed', 'perc_unemplyed', 'perc_unemployed_post_lockdown',
                'perc_have_jan_dhan', 'perc_need_med', 'perc_need_food', 'perc_no_benefit', 'perc_mgnrega', 
                'perc_pm_kisan', 'perc_ujjwala', 'perc_jan_dhan', 'perc_old_age_widow_disability',
                'perc_registered_labor', 'perc_free_ration']
state_dashboard = pd.DataFrame(columns = column_names)

In [21]:
for state in states:
    survey_state = survey[survey['state'] == state]
    date = survey_state['time_created'].max().strftime("%d/%m/%Y")
    num_surveyed = len(survey_state.shape)
    perc_unemployed = len(survey_state[(survey_state['employment_status'] == 'unemployed') | (survey_state['employment_status'] == 'unemployedPostLockdown')]) / num_surveyed
    perc_unemployed_post_lockdown = len(survey_state[survey_state['employment_status'] =='unemployedPostLockdown']) / num_surveyed
    perc_have_jan_dhan = len(survey_state[survey_state['eligible_jan_dhan'] == True]) / num_surveyed
    perc_need_med = len(survey_state[survey_state['need_meds'] == True]) / num_surveyed
    perc_need_food = len(survey_state[survey_state['have_food'] == False]) / num_surveyed
    perc_no_benefit = len(survey_state[survey_state['received_none'] == True]) / num_surveyed

    perc_mgnrega = np.NaN
    perc_pm_kisan = np.NaN
    perc_ujjwala = np.NaN
    perc_jan_dhan = np.NaN
    perc_old_age_widow_disability = np.NaN
    perc_registered_labor = np.NaN
    perc_free_ration = np.NaN

    if (len(survey_state[survey_state['eligible_mgnrega'] == True]) > 0):
        perc_mgnrega = len(survey_state[survey_state['received_mgnrega'] == True]) / len(survey_state[survey_state['eligible_mgnrega'] == True])

    if (len(survey_state[survey_state['eligible_pm_kisan'] == True]) > 0):
        perc_pm_kisan = len(survey_state[survey_state['received_pm_kisan'] == True]) / len(survey_state[survey_state['eligible_pm_kisan'] == True])

    if (len(survey_state[survey_state['eligible_ujjwala'] == True]) > 0):
        perc_ujjwala = len(survey_state[survey_state['received_ujjwala'] == True]) / len(survey_state[survey_state['eligible_ujjwala'] == True])

    if (len(survey_state[survey_state['eligible_jan_dhan'] == True]) > 0):
        perc_jan_dhan = len(survey_state[survey_state['received_jan_dhan'] == True]) / len(survey_state[survey_state['eligible_jan_dhan'] == True])

    if (len(survey_state[survey_state['eligible_old_age_widow'] == True]) > 0):
        perc_old_age_widow_disability = len(survey_state[survey_state['received_old_age_widow'] == True]) / len(survey_state[survey_state['eligible_old_age_widow'] == True])

    if (len(survey_state[survey_state['eligible_registered_labor'] == True]) > 0):
        perc_registered_labor = len(survey_state[survey_state['received_registered_labor'] == True]) / len(survey_state[survey_state['eligible_registered_labor'] == True])

    if (len(survey_state[(survey_state['eligible_bpl_ration'] == True) | (survey_state['eligible_general_ration'] == True)]) > 0):
        perc_free_ration = len(survey_state[survey_state['received_free_ration'] == True]) / len(survey_state[(survey_state['eligible_bpl_ration'] == True) | (survey_state['eligible_general_ration'] == True)])

    state_dashboard = state_dashboard.append(pd.Series([state, date, num_surveyed, perc_unemployed, 
             perc_unemployed_post_lockdown, perc_have_jan_dhan, perc_need_med, perc_need_food, 
             perc_no_benefit, perc_mgnrega, perc_pm_kisan, perc_ujjwala, perc_jan_dhan, 
             perc_old_age_widow_disability, perc_registered_labor, perc_free_ration], 
             index=state_dashboard.columns ), ignore_index=True)

In [22]:
state_dashboard

,state,date,num_surveyed,perc_unemplyed,perc_unemployed_post_lockdown,perc_have_jan_dhan,perc_need_med,perc_need_food,perc_no_benefit,perc_mgnrega,perc_pm_kisan,perc_ujjwala,perc_jan_dhan,perc_old_age_widow_disability,perc_registered_labor,perc_free_ration
0,Rajasthan,06/04/2020,2,1.0,0.0,1.0,0.0,0.0,2.0,0.0,0.000000,NaN,0.000000,NaN,NaN,0.000000
1,Karnataka,06/04/2020,2,0.0,0.0,1.0,0.0,0.0,1.0,NaN,NaN,NaN,0.000000,NaN,NaN,0.500000
2,Maharashtra,06/04/2020,2,0.5,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,0.000000
3,Chhattisgarh,06/04/2020,2,0.5,0.5,1.5,0.5,0.5,1.0,0.0,0.000000,0.0,0.333333,NaN,NaN,1.333333
4,Telangana,06/04/2020,2,0.5,0.5,0.0,0.5,0.0,0.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,Uttar Pradesh,06/04/2020,2,4.0,1.0,2.5,0.0,2.5,3.5,0.0,0.333333,0.0,0.600000,0.0,0.0,0.444444
6,Madhya Pradesh,06/04/2020,2,4.0,3.5,2.0,0.0,2.5,3.0,NaN,NaN,NaN,0.750000,NaN,NaN,0.200000
7,Delhi,06/04/2020,2,2.0,1.0,1.5,1.0,1.5,1.5,0.0,NaN,0.0,0.000000,NaN,NaN,0.400000
8,Haryana,06/04/2020,2,0.0,0.0,0.0,0.0,0.0,0.5,NaN,NaN,NaN,NaN,NaN,NaN,0.000000
